In [3]:
%pip install langchain langgraph cassio langchain_community langchain_groq tiktoken langchainhub chromadb langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.7 MB/s eta 0:00:0

In [4]:
import cassio
Astra_DB_APPLICATION_TOKEN="AstraCS:uqydRjIMfcykwFiRHscQoabi:8a8489394f0b12fc5b74fa1ddf3b1d0ffe4f901b06112db3d46d1ef960d1ea2b"

In [5]:
ASTRA_DB_ID="2cc60a90-0e5e-4281-8344-49313b1da310"

In [6]:
cassio.init(token=Astra_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma



urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs=[WebBaseLoader(url).load() for url in urls]
doc_list=[item for sublist in docs for item in sublist]
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
docs_split = text_splitter.split_documents(doc_list)



In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
from langchain.vectorstores.cassandra import Cassandra
astra_vector_store=Cassandra(embedding=embeddings,session=None,table_name='qa_mini_demo',keyspace=None)

In [10]:
%pip install langchain

In [12]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
astra_vector_store.add_documents(docs_split)
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [13]:
retriever = astra_vector_store.as_retriever()

In [14]:
retriever.invoke("what is Agent?")

[Document(id='4fba1179e7f54b4dba2e1e2551f74db3', metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory\n\nShort-term memory: I would consider all the in-context lear

In [15]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel,Field

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
class RouteQuery(BaseModel):
  datasource:Literal["vectorStore","wiki_search"]=Field(...,description="Given a question choose to route it to wikipediaor a vectorstore")

In [17]:
from langchain_groq import ChatGroq
from google.colab import userdata
import os

In [18]:
groq_api_key = userdata.get('groq_api_key')

In [19]:
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.3-70b-versatile")

In [20]:
structured_llm_router =llm.with_structured_output(RouteQuery)

In [21]:
system = """You are an expert at routing a user question to a vectorstore or wikipedia.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. Otherwise, use wiki-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

In [22]:
question_router = route_prompt | structured_llm_router

In [23]:
print(question_router.invoke({"question":"what is agent?"}))

datasource='vectorStore'


In [40]:
print(question_router.invoke({"question":"who is virat kholi?"}))

datasource='wiki_search'


In [24]:
%pip install wikipedia


In [26]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

In [28]:
api_wrapper= WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [30]:
wiki.run("tell me about generative ai")

'Page: ChatGPT\nSummary: ChatGPT is a generative artificial intelligence chatbot developed by the American company OpenAI and launched in 2022. It is based on large language models (LLMs) such as GPT-4o'

In [31]:
from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
  question:str
  generation:str
  documents:List[str]

In [32]:
from langchain.schema import document
def retrieve(state):
  print("--Retrieve--")
  questions = state["question"]

  documents =retriever.invoke(questions)
  return {"documents": documents,"question":questions}

In [50]:
from langchain.schema import document
from langchain_core.documents import Document

def wiki_search(state):
  print("--Wikipedia")
  print("--HELLO--")
  question = state["question"]
  print(question)

  docs = wiki.invoke({
      "query":question
  })
  wiki_results = docs

  if isinstance(wiki_results, str):
      wiki_results = Document(page_content=wiki_results)
  elif isinstance(wiki_results, list) and len(wiki_results) > 0 and isinstance(wiki_results[0], str):

       wiki_results = Document(page_content="\n".join(wiki_results))
  else:

      print(f"Unexpected type for wiki_results: {type(wiki_results)}. Attempting to convert to string.")
      wiki_results = Document(page_content=str(wiki_results))


  return {"documents":[wiki_results],"question":question}

In [51]:
def route_question(state):
    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})

    if hasattr(source, 'datasource'):
        if source.datasource == "wiki_search":
            print("---ROUTE QUESTION TO Wiki SEARCH---")
            return "wiki_search"
        elif source.datasource == "vectorstore":
            print("---ROUTE QUESTION TO RAG---")
            return "vectorstore"
        else:

            print(f"---UNEXPECTED DATASOURCE VALUE: {source.datasource}. Defaulting to vectorstore.---")
            return "vectorstore"

    else:

        print("---DATASOURCE ATTRIBUTE MISSING IN LLM OUTPUT. Defaulting to vectorstore.---")
        return "vectorstore"

In [52]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

workflow.add_node("wiki_search", wiki_search)
workflow.add_node("retrieve", retrieve)


workflow.add_conditional_edges(
    START,
    route_question,
    {
        "wiki_search": "wiki_search",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge( "retrieve", END)
workflow.add_edge( "wiki_search", END)

app = workflow.compile()

In [53]:
from pprint import pprint

inputs = {
    "question": "What is agent?"
}
for output in app.stream(inputs):
    for key, value in output.items():

        pprint(f"Node '{key}':")

    pprint("\n---\n")

pprint(value['documents'][0].dict()['metadata']['description'])

---ROUTE QUESTION---


---UNEXPECTED DATASOURCE VALUE: vectorStore. Defaulting to vectorstore.---
--Retrieve--
"Node 'retrieve':"
'\n---\n'
('Building agents with LLM (large language model) as its core controller is a '
 'cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer '
 'and BabyAGI, serve as inspiring examples. The potentiality of LLM extends '
 'beyond generating well-written copies, stories, essays and programs; it can '
 'be framed as a powerful general problem solver.\n'
 'Agent System Overview\n'
 'In a LLM-powered autonomous agent system, LLM functions as the agent’s '
 'brain, complemented by several key components:\n'
 '\n'
 'Planning\n'
 '\n'
 'Subgoal and decomposition: The agent breaks down large tasks into smaller, '
 'manageable subgoals, enabling efficient handling of complex tasks.\n'
 'Reflection and refinement: The agent can do self-criticism and '
 'self-reflection over past actions, learn from mistakes and refine them for '
 'future steps, thereby improving 

<ipython-input-53-a4ba987db3a3>:16: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  pprint(value['documents'][0].dict()['metadata']['description'])


In [62]:
from pprint import pprint


inputs = {
    "question": "Avengers"
}
for output in app.stream(inputs):
    for key, value in output.items():

        pprint(f"Node '{key}':")

    pprint("\n---\n")


pprint(value['documents'][0].page_content)

---ROUTE QUESTION---
---ROUTE QUESTION TO Wiki SEARCH---
--Wikipedia
--HELLO--
Avengers
"Node 'wiki_search':"
'\n---\n'
('Page: Avengers (comics)\n'
 'Summary: The Avengers are a team of superheroes appearing in American comic '
 'books published by Marvel Comics, created by writer-editor Stan Lee and '
 'artist/co-plotter Jack Kirby')
